In [1]:
!pip install --upgrade pythainlp
!pip install pyLDAvis
!pip install emoji

Requirement already up-to-date: pythainlp in /usr/local/lib/python3.6/dist-packages (2.2.6)


In [2]:
import pandas as pd
import pythainlp
import gensim
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
from pythainlp.corpus.common import thai_words
from pythainlp.tag import pos_tag
import emoji
import string

In [3]:
df = pd.read_csv('CustomerReviews.csv')
print(df.shape)
df.head()

(21, 7)


,Review ID,Restaurant_ID,Restaurant,User,Headline,Review,Rating
0,1,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,7b16469831074f7abc7824745ee75212,ที่สำคัญของร้านนี้คือบริการดีมากพนักงานน่ารักส...,ที่สำคัญของร้านนี้คือบริการดีมากพนักงานน่ารักส...,5.0
1,2,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,pakkaramonpondej,รสชาติเหมือนทุกสาขา แต่สาขานี้ บริการดี ที่นั่งดี,นึกถึงชาบูญี่ปุ่นยังไงก็ต้อง คิดถึงโมโม่ พาราไ...,5.0
2,3,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,saanowy,ชาบูพรีเมี่ยมสไตล์ญี่ปุ่น เนื้อดีมากกก,มาทานช่วงนี้ สามารถนั่งโต๊ะเดียวกัน หม้อเดียวก...,4.0
3,4,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ployynp,เนื้อดี ไอติมดี คุ้มค่าดี,ถ้านึกถึงชาบูที่มีเนื้อเน้นๆ ในราคาไม่โหดจนเกิ...,4.0
4,5,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,665a902b335b434ab489f00f2fbb477e,อาหารมีคุณภาพ บริการดีค่ะ,เดินมาหน้าร้านแล้วได้กลิ่นชาบูหอมมาก ๆ ประกอบ...,5.0


In [4]:
df.drop(columns = 'Restaurant_ID', inplace=True)
df.drop(columns = 'User', inplace=True)
df.drop(columns = 'Headline', inplace=True)
df.drop(columns = 'Rating', inplace=True)
df.drop(columns = 'Review ID', inplace=True)
df.head()

,Restaurant,Review
0,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ที่สำคัญของร้านนี้คือบริการดีมากพนักงานน่ารักส...
1,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,นึกถึงชาบูญี่ปุ่นยังไงก็ต้อง คิดถึงโมโม่ พาราไ...
2,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,มาทานช่วงนี้ สามารถนั่งโต๊ะเดียวกัน หม้อเดียวก...
3,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ถ้านึกถึงชาบูที่มีเนื้อเน้นๆ ในราคาไม่โหดจนเกิ...
4,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,เดินมาหน้าร้านแล้วได้กลิ่นชาบูหอมมาก ๆ ประกอบ...


In [5]:
stopwords = list(pythainlp.corpus.thai_stopwords())
removed_words = [' ', '  ', '\n', 'ร้าน', '(',')', '\u200b', 'ม', 'บาท', 'าน', 'เท', 'คนละ', 'กิน', 'ทาน', ',', '–', '/']
screening_words = stopwords + removed_words

def remove_emoji(text):
  return emoji.get_emoji_regexp().sub(u'', text)


def tokenize_with_space(sentence):
  merged = ''
  sentence = remove_emoji(sentence)
  custom_th_words = set(thai_words())
  FullWord = ['ชาบูชิ', 'กุ้งเทมปุระ', 'กุ้งสด', 'ขาบูชิ', 'ขาบู', 'เทมปุระ', 'บุฟเฟ่ต์', 'บุฟเฟ่', 'บุพเฟ่', 'โมโม', 'โมโม่',
               'ไอติม', 'ยูเนียนมอลล์', 'ยูเนี่ยนมอลล์', 'union mall', 'ญี่ปุ่น', 'หลากหลาย',
               'ซูชิ', 'กุ้งปลอกเปลือก', 'เดอะมอลล์', 'บางกะปิ', 'เดอะมอล', 'วิปครีม', 'พอนซึ', 'ราคา',
               'น้ำซุป', 'โชยุ', 'ไก่ไม้ไผ่', 'น้ำดำ', 'หอมมาก', 'พรีเมี่ยม', 'พรีเมียม', 'ญี่ปุน', 'ซีฟู้ด', 'ไปตำ', 'ไม่มีเหวี่ยง',
               'โมโมพาราไดส์', 'นารายพิซเซอเรีย', 'รอคิว', 'ฮันนี่โทส', 'น้ำซึป', 'กุ้งทอดเทมปุระ', 'บริการดี',
               'สันคอ', 'คุณภาพดี', 'รสชาต', 'โมโม่ พาราไดซ์', 'ไม่โอเค', 'ซอยซอส', 'เนื้อวัว', 'เนื้อหมู', 'เนื้อเป็ด',
               'โปรโมชั่น', 'ดังเดิม', 'โมโม – พาราไดส์', 'หมูสไลด์', 'เนื้อสไลด์', 'โอเด้ง', 'เบคอน', 'เบค่อน', 'หอยตลับ',
               'ตับหมู', 'บุฟเฟ', 'ชูชิ', 'รีฟีล', 'Nari pizzeria', 'สปาเกตตี้', 'ไก่นิวออลีน', 'งานดี', 'บราวนี่', 'บริการดี', 'ดูดี', 'ที่นั่ง']
  custom_th_words.update(FullWord)
  words = pythainlp.word_tokenize(str(sentence), engine = 'newmm')
  words_pos = pos_tag(words, corpus = 'orchid_ud')
  for word in words_pos:
    if word[0] not in screening_words:
      if word[1] in ['NOUN']:
        if not word[0].isnumeric():
          merged = merged + ',' + word[0]
  return merged[1:]

In [6]:
df['Review_tokenized'] = df['Review'].apply(lambda x: tokenize_with_space(x))

In [7]:
df_Momo = df[df['Restaurant'] == 'Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ']
df_Shabushi = df[df['Restaurant'] == 'Shabushi (ชาบูชิ) เดอะมอลล์บางกะปิ ชั้น G']
df_Shabu = df[df['Restaurant'] == 'ข้าน้อยขอชาบู']

In [8]:
df_Momo.head()

,Restaurant,Review,Review_tokenized
0,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ที่สำคัญของร้านนี้คือบริการดีมากพนักงานน่ารักส...,"พนักงาน,น่ารัก,สะอาดสะอ้าน,รสชาติ,อาหาร,อาหาร,..."
1,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,นึกถึงชาบูญี่ปุ่นยังไงก็ต้อง คิดถึงโมโม่ พาราไ...,"ชาบู,โม,โม่,พาราไดซ์,คุณภาพ,เค้า,มาตรา,ฐาน,มีโ..."
2,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,มาทานช่วงนี้ สามารถนั่งโต๊ะเดียวกัน หม้อเดียวก...,"โต๊ะ,หม้อ,โต๊ะ,ฉาก,กั้น,น้ำ,รส,น้ำจิ้ม,ตามน้ำ,..."
3,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ถ้านึกถึงชาบูที่มีเนื้อเน้นๆ ในราคาไม่โหดจนเกิ...,"ชาบู,เนื้อ,ราคา,โหด,โม,โม่,โปร,ส่วนตัว,เนื้อ,อ..."
4,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,เดินมาหน้าร้านแล้วได้กลิ่นชาบูหอมมาก ๆ ประกอบ...,"ชาบู,หอ,โปร,บัตรเครดิต,น้ำ,ชาบู,เนื้อ,ต่าง ๆ,ผ..."


In [9]:
df_Shabushi.head()

,Restaurant,Review,Review_tokenized
8,Shabushi (ชาบูชิ) เดอะมอลล์บางกะปิ ชั้น G,มา านที่ขาบูชิต้องมาตอนหิว ไม่งั้นจะไม่คุ้มนะค...,"ขา,บู,ชิ,ตอน,หิว,ไม่งั้น,ฮ่า,เมนู,ของกิน,อาหาร..."
9,Shabushi (ชาบูชิ) เดอะมอลล์บางกะปิ ชั้น G,ใครชอบกุ้งทอดเทมปุระ แค่กุ้งเทมปุระก็คุ้มแล้ว ...,"กุ้ง,ทอด,ปุระ,กุ้ง,ปุระ,ผิดหวัง,กุ้ง,ตัว,กลาย,..."
10,Shabushi (ชาบูชิ) เดอะมอลล์บางกะปิ ชั้น G,กลับมาอัพเดทราคาชาบูชิ ตอนนี้อยู่ที่ 399 บาท n...,"อัพเดท,ราคา,ชิ,net,ชม.,น้ำ,ซุป,รส,ซูชิ,ลายตา,ป..."
11,Shabushi (ชาบูชิ) เดอะมอลล์บางกะปิ ชั้น G,ห่างหายไปนานสำหรับชาบูชิ ตั้งแต่ทางร้านได้ปรับ...,"ชาบู,ชิ,ขึ้นราคา,แถม,เวลา,คุณภาพ,อาหาร,แย่,ได้..."
12,Shabushi (ชาบูชิ) เดอะมอลล์บางกะปิ ชั้น G,เมื่อหลายวันก่อนนัดหาข้าวทานกับเดอะแกงค์ และก็...,"วันก่อน,ข้าว,เดอะ,แกงค์,อาหาร,บุฟเฟ่ต์,อยุ่,ชั..."


In [10]:
df_Shabu.head()

,Restaurant,Review,Review_tokenized
16,ข้าน้อยขอชาบู,หลังจากที่เคยลองสาขายูเนี่ยนมอลล์ไป รอบนี้มาที...,"สาขา,ยู,นม,อลล์,รอบ,สาขา,เดอะ,มอลล์,กะปิ,ชั้น,..."
17,ข้าน้อยขอชาบู,ร้านบุฟเฟ่ต์ที่มีโปรโมชั่นหัวละ199บาท ไม่รวมน้...,"บุฟเฟ่ต์,โปรโมชั่น,หัว,น้ำ,VAT,ได้เวลา,ชม.,ค่า..."
18,ข้าน้อยขอชาบู,เคยรู้จักร้านนี้จากที่ union mall ไม่เคยได้ลอง...,"union,mall,จำได้,ชื่อ,ชื่อ,ชาบู,Narai,pizzeria..."
19,ข้าน้อยขอชาบู,อร่อยมากค่ะ คุ้มค่าสมราคา บุฟเฟ่หมูผักต่างๆ รว...,"อร่อย,คุ้มค่า,สมราคา,บุ,ฟเฟ่,หมู,ผัก,น้ำ,เด็ด,..."
20,ข้าน้อยขอชาบู,ก็ตามที่เขียนเลยครับ ว่า ถ้าจะจ่ายในราคา 199 บ...,"ราคา,เงินสด,ราคา,น้ำ,VAT,%,เนื้อวัว,ซูชิ,ของหว..."


### Mo-Mo-Paradise

In [11]:
Momo_documents = df_Momo['Review_tokenized'].to_list()
Momo_texts = [[text for text in doc.split(',')] for doc in Momo_documents]
Momo_dictionary = gensim.corpora.Dictionary(Momo_texts)

In [12]:
print(Momo_dictionary.token2id.keys())

dict_keys(['ข้างนอก', 'ความต้องการ', 'คอ', 'คุณภาพ', 'คุ้ม', 'ซุป', 'ถาด', 'น่ารัก', 'น้ำ', 'น้ำจิ้ม', 'บริการ', 'ประทับใจ', 'พนักงาน', 'ภาพ', 'รสชาติ', 'ราคา', 'สะอาดสะอ้าน', 'สัน', 'หลากหลาย', 'อร่อย', 'อาหาร', 'เนื้อ', 'เป็ด', 'เผ็ด', 'แฟน', 'โมจิ', '           ', 'CTW', 'Terminal', 'กา', 'การงด', 'ชาบู', 'ฐาน', 'ที่ทาง', 'นาที', 'พนักงานบริการ', 'พระราม', 'พรีเมียม', 'พาราไดซ์', 'มาตรา', 'มีโอ', 'รายละเอียด', 'ลูกค้า', 'สมาทาน', 'สาขา', 'ออร์เดอร์', 'เค้า', 'เหวี่ยง', 'แจ้ง', 'โม', 'โม่', '++', '::', 'กรุบ', 'กั้น', 'ของหวาน', 'คุ', 'จืด', 'ฉาก', 'ชาเขียว', 'ชิ้น', 'ซอย', 'ซอส', 'ตะ', 'ตามน้ำ', 'ถั่ว', 'นุ่ม', 'บด', 'บาร์', 'บู', 'ผัก', 'ภาษา', 'มีชื่อ', 'รส', 'รี่', 'ลอง', 'ลิ้น', 'ลืม', 'ววว', 'สด', 'สตอ', 'หม้อ', 'หวาน', 'เค็ม', 'เนื้อวัว', 'เนื้อสัตว์', 'เนื้อหมู', 'เป็นผู้ใหญ่', 'เสิร์ฟ', 'เห็ด', 'แดง', 'แตา', 'แนว', 'แย้', 'โควิด', 'โต๊ะ', 'โร', 'ไก่', 'ไข่', 'ไดฟูกุ', 'ไผ่', 'ไม้', 'ไอติม', 'ข้าว', 'ถ้วย', 'ส', 'ส่วนตัว', 'ออ', 'โปร', 'โหด', 'ไว', 'ไอศกรีม', 'ขนม', 'คุณภาพดี

In [13]:
Momo_gensim_corpus = [Momo_dictionary.doc2bow(text, allow_update=True) for text in Momo_texts]
Momo_word_frequencies = [[(Momo_dictionary[id], frequence) for id, frequence in couple] for couple in Momo_gensim_corpus]

In [14]:
Momo_num_topics = 4
Momo_chunksize = 21 # size of the doc looked at every pass
Momo_passes = 20 # number of passes through documents
Momo_iterations = 50
Momo_eval_every = 1  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
Momo_temp = Momo_dictionary[0]  # This is only to "load" the dictionary.
Momo_id2word = Momo_dictionary.id2token

%time Momo_model = gensim.models.LdaModel(corpus=Momo_gensim_corpus, id2word=Momo_id2word, chunksize=Momo_chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=Momo_iterations, num_topics=Momo_num_topics, \
                       passes=Momo_passes, eval_every=Momo_eval_every)

CPU times: user 116 ms, sys: 0 ns, total: 116 ms
Wall time: 116 ms


In [15]:
pyLDAvis.gensim.prepare(Momo_model, Momo_gensim_corpus, Momo_dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.046776 -0.006896       1        1  48.125913
0     -0.038659 -0.015414       2        1  41.379997
2     -0.009893  0.026820       3        1  10.451726
1      0.001776 -0.004509       4        1   0.042363, topic_info=       Term       Freq      Total Category  logprob  loglift
8       น้ำ  14.000000  14.000000  Default  30.0000  30.0000
31     ชาบู  11.000000  11.000000  Default  29.0000  29.0000
21    เนื้อ  10.000000  10.000000  Default  28.0000  28.0000
14   รสชาติ   7.000000   7.000000  Default  27.0000  27.0000
19    อร่อย   6.000000   6.000000  Default  26.0000  26.0000
..      ...        ...        ...      ...      ...      ...
102   ไอติม   0.001309   3.160219   Topic4  -4.5654  -0.0223
149     หมู   0.001013   2.269181   Topic4  -4.8219   0.0524
108     โปร   0.001007   2.261527   Topic4  -4.8280   0.0497
88   เสิร์ฟ   0.000894   2.680611   Topic4  -4.9466  -0.2389
82     หวาน   0.000880   2.663388   Topic4  -4.9623  -0.2481

[256 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
52        1  0.523421        ::
27        1  0.526074       CTW
27        2  0.526074       CTW
133       2  0.880062    Number
134       2  0.882783  กรรมวิธี
...     ...       ...       ...
101       1  0.521979       ไม้
110       1  0.914527        ไว
102       1  0.632868     ไอติม
102       2  0.316434     ไอติม
111       1  0.914482   ไอศกรีม

[152 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 1, 3, 2])

In [16]:
Momo_model.show_topic(0)

[('เนื้อ', 0.03971864),
 ('น้ำ', 0.039601985),
 ('ชาบู', 0.03819904),
 ('รสชาติ', 0.027342858),
 ('อร่อย', 0.026168203),
 ('น้ำจิ้ม', 0.026126133),
 ('โม', 0.025754957),
 ('อาหาร', 0.022826701),
 ('สาขา', 0.020762324),
 ('ผัก', 0.018132538)]

In [17]:
df_Momo['topics'] = df_Momo['Review_tokenized'].apply(lambda x: Momo_model.get_document_topics(Momo_dictionary.doc2bow(x.split(',')))[0][0])
df_Momo['score'] = df_Momo['Review_tokenized'].apply(lambda x: Momo_model.get_document_topics(Momo_dictionary.doc2bow(x.split(',')))[0][1])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [18]:
df_Momo.head()

,Restaurant,Review,Review_tokenized,topics,score
0,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ที่สำคัญของร้านนี้คือบริการดีมากพนักงานน่ารักส...,"พนักงาน,น่ารัก,สะอาดสะอ้าน,รสชาติ,อาหาร,อาหาร,...",0,0.997033
1,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,นึกถึงชาบูญี่ปุ่นยังไงก็ต้อง คิดถึงโมโม่ พาราไ...,"ชาบู,โม,โม่,พาราไดซ์,คุณภาพ,เค้า,มาตรา,ฐาน,มีโ...",2,0.995527
2,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,มาทานช่วงนี้ สามารถนั่งโต๊ะเดียวกัน หม้อเดียวก...,"โต๊ะ,หม้อ,โต๊ะ,ฉาก,กั้น,น้ำ,รส,น้ำจิ้ม,ตามน้ำ,...",3,0.998839
3,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ถ้านึกถึงชาบูที่มีเนื้อเน้นๆ ในราคาไม่โหดจนเกิ...,"ชาบู,เนื้อ,ราคา,โหด,โม,โม่,โปร,ส่วนตัว,เนื้อ,อ...",3,0.993856
4,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,เดินมาหน้าร้านแล้วได้กลิ่นชาบูหอมมาก ๆ ประกอบ...,"ชาบู,หอ,โปร,บัตรเครดิต,น้ำ,ชาบู,เนื้อ,ต่าง ๆ,ผ...",0,0.995492


### Shabushi

In [19]:
Shabushi_documents = df_Shabushi['Review_tokenized'].to_list()
Shabushi_texts = [[text for text in doc.split(',')] for doc in Shabushi_documents]
Shabushi_dictionary = gensim.corpora.Dictionary(Shabushi_texts)

In [20]:
print(Shabushi_dictionary.token2id.keys())

dict_keys(['กุ้ง', 'ของกิน', 'ขา', 'คุ้ม', 'ชิ', 'ซูชิ', 'ซ่า', 'ตอน', 'บู', 'ปุระ', 'ปูอัด', 'ราคา', 'สายพาน', 'หิว', 'อาหาร', 'ฮ่า', 'เกี๊ยว', 'เมนู', 'แพง', 'ไม่งั้น', 'กลาย', 'ชาบู', 'ชื่นใจ', 'ญี่', 'ตัว', 'ทอด', 'นพ', 'ปลา', 'ปุ', 'ผิดหวัง', 'ลืม', 'วก', 'อาหารเสริม', 'เป็นมา', 'เป็นหลัก', 'เย็น', 'ไข่', 'net', 'ของคาว', 'ชม.', 'ซุป', 'น้ำ', 'บุฟเฟ่ต์', 'รส', 'ลายตา', 'สด', 'สไลด์', 'อัพเดท', 'อิ่ม', 'เนื้อ', 'เบ', 'เพลิน', 'ไลน์', '/คน', 'G', 'Net', 'Shabushi', 'กรอ', 'กะปิ', 'ของหวาน', 'ขาดสารอาหาร', 'ขึ้นราคา', 'ข่าว', 'คน', 'คนเดียว', 'คิว', 'คุณภาพ', 'งาน', 'จาน', 'ชั่วโมง', 'ชั้น', 'ชื่น', 'ตลับ', 'ตับ', 'ถูกใจ', 'นาที', 'นุ่ม', 'น้ำมัน', 'ปลาดิบ', 'ปอกเปลือก', 'ฝรั่ง', 'พนักงาน', 'มอลล์', 'มื้อ', 'รสชาติ', 'ระดับ', 'รายการ', 'ลูก', 'ลูกชิ้น', 'วันหยุด', 'วานิลา', 'สะอาด', 'สาขา', 'สาเหตุ', 'สไปรท์', 'ส่วนตัว', 'หมู', 'หวาน', 'หอม', 'หอย', 'หาไม่', 'ห้าง', 'อย่า', 'อร่อย', 'อาทิตย์', 'เดอะ', 'เด้ง', 'เบคอน', 'เปรี้ยวหวาน', 'เพื่อน', 'เรื่อง', 'เวลา', 'เสาร์', 'แก้ว', 'แถม',

In [21]:
Shabushi_gensim_corpus = [Shabushi_dictionary.doc2bow(text, allow_update=True) for text in Shabushi_texts]
Shabushi_word_frequencies = [[(Shabushi_dictionary[id], frequence) for id, frequence in couple] for couple in Shabushi_gensim_corpus]

In [22]:
Shabushi_documents = df_Shabushi['Review_tokenized'].to_list()
Shabushi_texts = [[text for text in doc.split(',')] for doc in Shabushi_documents]
Shabushi_dictionary = gensim.corpora.Dictionary(Shabushi_texts)

In [23]:
Shabushi_num_topics = 3
Shabushi_chunksize = 21 # size of the doc looked at every pass
Shabushi_passes = 20 # number of passes through documents
Shabushi_iterations = 50
Shabushi_eval_every = 1  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
Shabushi_temp = Shabushi_dictionary[0]  # This is only to "load" the dictionary.
Shabushi_id2word = Shabushi_dictionary.id2token

%time Shabushi_model = gensim.models.LdaModel(corpus=Shabushi_gensim_corpus, id2word=Shabushi_id2word, chunksize=Shabushi_chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=Shabushi_iterations, num_topics=Shabushi_num_topics, \
                       passes=Shabushi_passes, eval_every=Shabushi_eval_every)

CPU times: user 143 ms, sys: 2.18 ms, total: 145 ms
Wall time: 146 ms


In [24]:
pyLDAvis.gensim.prepare(Shabushi_model, Shabushi_gensim_corpus, Shabushi_dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.024259 -0.005783       1        1  47.180306
1      0.022265 -0.007489       2        1  39.256685
2      0.001993  0.013272       3        1  13.563009, topic_info=     Term       Freq      Total Category  logprob  loglift
0    กุ้ง  11.000000  11.000000  Default  30.0000  30.0000
9    ปุระ   5.000000   5.000000  Default  29.0000  29.0000
21   ชาบู   4.000000   4.000000  Default  28.0000  28.0000
4      ชิ   7.000000   7.000000  Default  27.0000  27.0000
17   เมนู   4.000000   4.000000  Default  26.0000  26.0000
..    ...        ...        ...      ...      ...      ...
105  เดอะ   0.439729   4.007175   Topic3  -4.6931  -0.2119
76   นุ่ม   0.420291   3.938212   Topic3  -4.7383  -0.2397
41    น้ำ   0.343980   3.065141   Topic3  -4.9386  -0.1894
54      G   0.343780   3.107012   Topic3  -4.9392  -0.2036
70   ชั้น   0.342800   3.101451   Topic3  -4.9421  -0.2047

[204 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
54        1  0.643705         G
54        2  0.321853         G
56        1  0.520266  Shabushi
58        1  0.646518      กะปิ
58        2  0.323259      กะปิ
...     ...       ...       ...
120       1  0.883445      โฟลต
197       2  0.917083    โออิชิ
122       1  0.520307      โอเค
19        2  0.939719   ไม่งั้น
52        2  0.928422      ไลน์

[150 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3])

In [36]:
Shabushi_model.show_topic(1)

[('อาหาร', 0.028772822),
 ('กุ้ง', 0.026616663),
 ('ชิ', 0.024289198),
 ('ราคา', 0.019701453),
 ('สายพาน', 0.018533856),
 ('คน', 0.017101964),
 ('ซูชิ', 0.01667522),
 ('ปุระ', 0.015925972),
 ('เนื้อ', 0.014742171),
 ('เมนู', 0.014521734)]

### ข้าน้อยขอชาบู

In [26]:
Shabu_documents = df_Shabu['Review_tokenized'].to_list()
Shabu_texts = [[text for text in doc.split(',')] for doc in Shabu_documents]
Shabu_dictionary = gensim.corpora.Dictionary(Shabu_texts)

In [27]:
print(Shabu_dictionary.token2id.keys())

dict_keys(['HALL', 'MCC', 'กะปิ', 'ของหวาน', 'ค่ายมวย', 'ชั้น', 'ชาบู', 'ซุป', 'ซูชิ', 'ด้านใน', 'ตอนแรก', 'ตัวเลือก', 'ทอด', 'ที่ไหนได้', 'นม', 'นา', 'น้ำ', 'น้ำดื่ม', 'พิซ', 'พิซซ่า', 'ฟี', 'มอลล์', 'ยู', 'รสชาติ', 'รอบ', 'ราคา', 'รี', 'ล', 'สาขา', 'หมู', 'หิว', 'อลล์', 'อเรีย', 'เซ', 'เดอะ', 'เนื้อ', 'เบ', 'เมนู', 'แกงเขียวหวาน', 'โซน', 'VAT', 'ค่าน้ำ', 'ชม.', 'ชุด', 'ช่อง', 'ซี', 'ซึป', 'ต้มยำ', 'ทอน', 'บุฟเฟ่ต์', 'ฟรี', 'ฟู้ด', 'ลอตเตอรี่', 'สัปดาห์', 'หม้อ', 'หัว', 'อันนี้', 'อาหาร', 'ฮัน', 'เงิน', 'โทส', 'โปรโมชั่น', 'ได้เวลา', 'Narai', 'mall', 'pizza', 'pizzeria', 'union', 'ขยัน', 'ของสด', 'ข้าวมันไก่', 'คุณภาพ', 'จำได้', 'ชื่อ', 'ตตี้', 'ตัดกำลัง', 'ตู้', 'น', 'นิว', 'บางส่วน', 'ปรุง', 'พนักงาน', 'ลี', 'สปา', 'สุก', 'อร่อย', 'ออ', 'เก', 'ไก่', 'ไลน์', 'กก', 'ครีม', 'คุ้มค่า', 'งาน', 'ชาต', 'นุ่ม', 'บรา', 'บุ', 'บุพ', 'ผัก', 'ฟเฟ่', 'รส', 'ว', 'วิป', 'สมราคา', 'หอ', 'เซต', 'เด็ด', 'เป้ง', 'เฟ่', 'เรย', 'เเบบ', 'โชค', 'ไอติม', '%', 'ตอน', 'สดๆ ร้อนๆ', 'หน้า', 'เงินสด', 'เนื้อวัว

In [28]:
Shabu_gensim_corpus = [Shabu_dictionary.doc2bow(text, allow_update=True) for text in Shabu_texts]
Shabu_word_frequencies = [[(Shabu_dictionary[id], frequence) for id, frequence in couple] for couple in Shabu_gensim_corpus]

In [29]:
Shabu_documents = df_Shabu['Review_tokenized'].to_list()
Shabu_texts = [[text for text in doc.split(',')] for doc in Shabu_documents]
Shabu_dictionary = gensim.corpora.Dictionary(Shabu_texts)

In [30]:
Shabu_num_topics = 3
Shabu_chunksize = 21 # size of the doc looked at every pass
Shabu_passes = 20 # number of passes through documents
Shabu_iterations = 50
Shabu_eval_every = 1  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
Shabu_temp = Shabu_dictionary[0]  # This is only to "load" the dictionary.
Shabu_id2word = Shabu_dictionary.id2token

%time Shabu_model = gensim.models.LdaModel(corpus=Shabu_gensim_corpus, id2word=Shabu_id2word, chunksize=Shabu_chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=Shabu_iterations, num_topics=Shabu_num_topics, \
                       passes=Shabu_passes, eval_every=Shabu_eval_every)

CPU times: user 108 ms, sys: 684 µs, total: 109 ms
Wall time: 115 ms


In [31]:
pyLDAvis.gensim.prepare(Shabu_model, Shabu_gensim_corpus, Shabu_dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.024657 -0.002592       1        1  57.205667
0      0.015177 -0.015529       2        1  21.974943
1      0.009480  0.018121       3        1  20.819391, topic_info=        Term      Freq     Total Category  logprob  loglift
49  บุฟเฟ่ต์  2.000000  2.000000  Default  30.0000  30.0000
16       น้ำ  7.000000  7.000000  Default  29.0000  29.0000
85     อร่อย  4.000000  4.000000  Default  28.0000  28.0000
3    ของหวาน  4.000000  4.000000  Default  27.0000  27.0000
43       ชุด  1.000000  1.000000  Default  26.0000  26.0000
..       ...       ...       ...      ...      ...      ...
57     อาหาร  0.779445  3.784233   Topic3  -3.8332  -0.0107
19    พิซซ่า  1.000681  4.899448   Topic3  -3.5833  -0.0192
85     อร่อย  0.740109  4.491817   Topic3  -3.8850  -0.2339
25      ราคา  0.840123  6.139663   Topic3  -3.7582  -0.4197
3    ของหวาน  0.735792  4.474547   Topic3  -3.8908  -0.2359

[161 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
114       1  0.917184          %
0         1  0.882486       HALL
1         1  0.883210        MCC
63        1  0.859271      Narai
40        1  0.489173        VAT
...     ...       ...        ...
61        3  0.528912  โปรโมชั่น
121       1  0.917778       โอเค
88        1  0.471702        ไก่
88        2  0.471702        ไก่
89        1  0.857696       ไลน์

[94 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 2])

In [32]:
Shabu_model.show_topic(1)

[('น้ำ', 0.050812922),
 ('บุฟเฟ่ต์', 0.03508341),
 ('พิซซ่า', 0.027783142),
 ('ราคา', 0.023325376),
 ('ซุป', 0.02174706),
 ('อาหาร', 0.021640703),
 ('อร่อย', 0.02054856),
 ('ของหวาน', 0.020428708),
 ('ชุด', 0.01902054),
 ('โปรโมชั่น', 0.015799187)]